# Numeric Representation of Text


In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import Dataset

In [11]:
import json
# Load the json data

with open("../fact_checking_data//fact_checking_data_extracted.json", 'r', encoding='utf-8') as file:
    json_data = json.load(file)  

### 1. Lowercasing the Text and Mapping the Labels


In [22]:
label_map = {
    "verdadero": 0,
    "falso": 1,
    "impreciso": 2,
    "engañoso": 3,
    "se puso creativ@": 4,
    "creíble": 5,
    "sería creíble, pero...": 6,
    "creíble, pero...": 7,
    "no es creíble": 8,
    "ciencia ficción": 9,
}

fact_data = {"texts": [], "labels": []}

for item in json_data["facts"]:
    if item["veracity"].lower() in label_map:
        fact_data["texts"].append(item["content"].lower())
        fact_data["labels"].append(label_map[item["veracity"].lower()])

### 2. Tokenize the Text and Remove Stop Words


In [4]:
# from transformers import pipeline
# from transformers import AutoTokenizer, AutoModelForMaskedLM

# pipe = pipeline("fill-mask", model="dccuchile/bert-base-spanish-wwm-uncased")

# def tokenize_function(example):
#     # print(example["text"])
#     # filtered_text = ' '.join([word for word in example["text"].split(])
#     # print(filtered_text)
#     return tokenizer(example["text"], padding="max_length", truncation=True)

# tokenized_dataset = dataset.map(tokenize_function)
# tokenized_dataset = tokenized_dataset.map(lambda x: {"label": int(x["label"])})


BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Map:   0%|          | 0/139 [00:00<?, ? examples/s]

Map:   0%|          | 0/139 [00:00<?, ? examples/s]

In [28]:
# print(len(tokenized_dataset["text"]))

139


### 5. Modeling


In [58]:
# Dividir los datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
train_texts, test_texts, train_labels, test_labels = train_test_split(fact_data['texts'], fact_data['labels'], test_size=0.2)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        if idx >= len(self.labels):  # Validación del índice
            raise IndexError(f"Índice {idx} fuera de rango para el dataset de longitud {len(self.labels)}")
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

batch_size = 8
num_train_examples = len(train_dataset)

if num_train_examples % batch_size != 0:
    print(f"Ajustando el número de ejemplos en el dataset de entrenamiento.")
    train_labels = train_labels[:-(num_train_examples % batch_size)]
    train_encodings = {k: v[:-(num_train_examples % batch_size)] for k, v in train_encodings.items()}
    train_dataset = TextDataset(train_encodings, train_labels)

# Configuración de los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Entrenar el modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ajustando el número de ejemplos en el dataset de entrenamiento.


  0%|          | 0/39 [00:00<?, ?it/s]

IndexError: index out of range in self

In [50]:
print(f"Longitud de train_encodings['input_ids']: {len(train_encodings['input_ids'])}")
print(f"Longitud de train_labels: {len(train_labels)}")

for key, value in train_encodings.items():
    print(f"{key}: {len(value)}")

Longitud de train_encodings['input_ids']: 111
Longitud de train_labels: 111
input_ids: 111
token_type_ids: 111
attention_mask: 111
